### Test the processImage pipeline on a single image

In [15]:
TEST_IMAGE_PATH = "test_images/challenge3.png"
import os
imgs = os.listdir("test_images/")
imgs = ["test_images/" + x for x in  imgs ]
IMAGES = [TEST_IMAGE_PATH, "all"]
IMAGES = IMAGES + imgs

hyperParamsDefault = {
    "blurKernalSize": (5,1,29,2),  # (value, min, max, step)
    "cannyHT": (300,0,500,10),
    "cannyLT": (50,0,500,10),
    "bThresh": (200,0,255,1),
    "hThresh": (10,0,100,1),
    "hMinLineLen": (20,0,100,1),
    "hMaxLineGap": (1.0,0.0,40.0,0.1),
    "useBrightness": (True),
    "images": IMAGES,
    "outputStage": ["final", "final", "gray", "blur", "canny", "hsv", "hue", "saturation", "brightness", "bMask", "hough", "linedImg"]
}


In [16]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import helpers
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

%matplotlib inline  
%load_ext autoreload
%autoreload 2

from helpers import *
from lanedetector import detectLanes
from io import BytesIO
from base64 import b64encode
from PIL import Image
from IPython.core.display import display, HTML


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
# Disable Autoscrolling
from IPython.display import display, Javascript

disable_js = """
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
"""

def load_ipython_extension(ip):
    display(Javascript(disable_js))
    print ("autoscrolling long output is disabled")

load_ipython_extension(0)

<IPython.core.display.Javascript object>

autoscrolling long output is disabled


In [18]:

def parseParams(hyParams):
    outP = {}
    for k in hyParams:
        try:
            outP[k] = hyParams[k][0]
        except:
            outP[k] = hyParams[k]
    return outP

def parseParamsInteract(hyParams):
    outP = {}
    for k in hyParams:
        try:
            outP[k] = (hyParams[k][1:])
        except:
            outP[k] = hyParams[k]
    return outP
        
hyperParams = parseParams(hyperParamsDefault)
hyperParamsInteract = parseParamsInteract(hyperParamsDefault)

In [19]:
# InLine image display helpers

def getPngBytes(img):
    pil_im = Image.fromarray(img)
    b = BytesIO()  
    pil_im.save(b, format='png')
    return b.getvalue()
    
def displaySideBySide(im1, im2):
    
    im1 = cv2.cvtColor(im1, cv2.COLOR_BGR2RGB)
    b1 = getPngBytes(im1)
    b2 = getPngBytes(im2)
    b641 = b64encode(b1).decode('utf-8')
    b642 = b64encode(b2).decode('utf-8')
    HTMLDATA = "<table><tr><td><img src='data:image/png;base64,{0}'/></td><td><img src='data:image/png;base64,{1}'/></td></tr></table>".format(b641, b642)
    display(HTML(HTMLDATA))

In [28]:
# Interactive parameter tuning

PLAYED_PARAMS = {}

def assignDefaults(wids, defVal):
    valDict = {}
    for wid in wids.children:
        try:
            valDict[wid.description] = wid
        except:
            pass
    for k in valDict:
        try:
            valDict[k].value = defVal[k]
        except:
            pass

def processImgPath(imgPath, kwargs):
    image = readImg(imgPath)
    outP =(detectLanes(image, kwargs))
    displaySideBySide(image, outP)

def playWithParams(**kwargs):
    global PLAYED_PARAMS
    PLAYED_PARAMS = kwargs
    if 'prevLines' in kwargs: del kwargs['prevLines']
    try:
        if kwargs["images"] == "all":
            for imgPath in IMAGES[2:]:
                if 'prevLines' in kwargs: del kwargs['prevLines']
                processImgPath(imgPath, kwargs)
        else:
            if 'prevLines' in kwargs: del kwargs['prevLines']
            processImgPath(kwargs["images"], kwargs)
            
    except:
        pass
        #print "EXCEPTION!", e
        #outP = image
    
w = interactive(playWithParams, **hyperParamsInteract)

import json
with open('params.json', 'r') as fp:
    global hyperParams
    try:
        hyperParams = json.load(fp)
    except:
        print("Failed loading params from params.json file!")
    
assignDefaults(w, hyperParams)
display(w)


A Jupyter Widget

In [31]:
PLAYED_PARAMS

{'bThresh': 220,
 'blurKernalSize': 9,
 'cannyHT': 270,
 'cannyLT': 60,
 'hMaxLineGap': 25.2,
 'hMinLineLen': 9,
 'hThresh': 27,
 'images': 'all',
 'maskLineThickness': 8,
 'minY': 450,
 'outputStage': 'final',
 'useBrightness': True}

In [33]:
import json

def saveParams():
    with open('params.json', 'w') as fp:
        json.dump(PLAYED_PARAMS, fp)
        
    print("Params saved to params.json!")
# Call this to save the params
saveParams()

Params saved to params.json!
